In [1]:
import findspark
findspark.init()

import json
import re
import numpy as np
import math
from pyspark import SparkContext
from collections import Counter

#trial
from datetime import datetime
start = datetime.now()
sc = SparkContext()

/home/rutvij/Spark_Installation/spark-3.0.0-preview2-bin-hadoop3.2/python/pyspark/context.py:219: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [2]:
user_id = 50

In [20]:
# reading user vectors
import pickle
filename = "item_user_rating.json"
with open(filename, 'rb') as fp:
    r = pickle.load(fp)
user_item_rating = sc.parallelize(r).filter(lambda x: x[0][0] == user_id)

In [68]:
# storing items of user_id in broadcast variable
user_id_items = sc.broadcast(set(user_item_rating.map(lambda x: x[0][1]).collect()))
# user_item_rating.take(1)

In [5]:
# reading item vectos
filename = "dummy_data_v2.json"

def convert(item_vector):

    item_id = item_vector["ItemID"]
    features = []
    
    for feature, value in item_vector.items():
        if feature == "ItemID":
            continue
        features.append((feature, value))
        
    return (item_id, features)

rdd = sc.textFile(filename).map(lambda x: json.loads(x)).map(lambda x: convert(x))

In [43]:
# normalize item vectors
def normalize_vector(v):
#     item = v[0]
#     features = v[1]    
#     mean_rating = sum(map(lambda x: x[1], features)) / len(features)
#     normalized_vector = list(map(lambda x: (x[0], x[1] - mean_rating), features))
    
    item = v[0]
    features = v[1]    
    mean_rating = sum(map(lambda x: x[1], features)) / len(features)
    normalized_vector = list(map(lambda x: (x[0], x[1] / mean_rating), features))
    
    return (item, normalized_vector)
normalized_rdd = rdd.map(lambda x: normalize_vector(x)).flatMapValues(list)
# o/p -> (item, (feature, rating))

In [44]:
user_item_rating.take(1)

[((50, 9793), 3)]

In [45]:
normalized_rdd.take(1)

[(1, ('myntraRating', 0.0015706173734444846))]

In [46]:
item_user_rating = user_item_rating.map(lambda x: (x[0][1], (x[0][0], x[1])))
item_user_rating.take(1)
# o/p -> (item, (user, rating))

[(9793, (50, 3))]

In [47]:
joined_rdd = normalized_rdd.join(item_user_rating)
joined_rdd.take(1)
# o/p -> (item, ((feature, value), (user, rating)))

[(8610, (('myntraRating', 0.0), (50, 2)))]

In [48]:
new_joined = joined_rdd.map(lambda x: (x[1][0][0], (x[0], x[1][1][0], x[1][0][1] * x[1][1][1])))
new_joined.take(1)

[('myntraRating', (8610, 50, 0.0))]

In [73]:
new_joined_r = new_joined.reduceByKey(lambda x, y: (x[0], x[1], x[2] * y[2])).map(lambda x: (x[0], (x[1][1], x[1][2])))
new_joined_r.take(1)

[('usage', (50, 0.0))]

In [65]:
new_joined_r.count()

13

In [69]:
item_ratings = normalized_rdd.filter(lambda x: x[0] not in user_id_items.value)\
                                .map(lambda x: (x[1][0], (x[0], x[1][1])))
                                

In [75]:
item_ratings.take(15)

[('myntraRating', (1, 0.0015706173734444846)),
 ('productDisplayName', (1, 11.189078168418508)),
 ('discountedPrice', (1, 1.0350368490999153)),
 ('gender', (1, 0.003141234746888969)),
 ('baseColour', (1, 0.014135556361000362)),
 ('season', (1, 0.003141234746888969)),
 ('brandName', (1, 0.19004470218678263)),
 ('year', (1, 0.007853086867222423)),
 ('usage', (1, 0.0015706173734444846)),
 ('productTypeId', (1, 0.04711852120333454)),
 ('price', (1, 0.49160323788812366)),
 ('styleType', (1, 0.014135556361000362)),
 ('fashionType', (1, 0.0015706173734444846)),
 ('myntraRating', (2, 0.0)),
 ('productDisplayName', (2, 12.46316283435007))]

In [76]:
rec_join = item_ratings.join(new_joined_r)

In [77]:
rec_join.take(1)

[('styleType', ((1, 0.014135556361000362), (50, 2.425951264341751e-247)))]

In [80]:
rec_join.map(lambda x: ((x[1][1][0], x[1][0][0]), (x[1][0][1] * x[1][1][1]))).reduceByKey(lambda x,y: x + y).top(10, lambda x: x[1])

[((50, 14175), 7.985869503227364e+224),
 ((50, 22393), 7.98553153083483e+224),
 ((50, 37904), 7.984831626193657e+224),
 ((50, 25620), 7.982062410487253e+224),
 ((50, 32034), 7.977555967448369e+224),
 ((50, 29057), 7.974673163943141e+224),
 ((50, 3244), 7.967290273192288e+224),
 ((50, 2888), 7.965397795584064e+224),
 ((50, 28287), 7.96503370730799e+224),
 ((50, 25291), 7.96503370730799e+224)]

In [81]:
14175 in user_id_items.value

False